In [1]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
os.environ['MUJOCO_PY_MUJOCO_PATH']='/scratch_net/biwidl211/rl_course_10/.mujoco/mujoco210'

#add DiffuseDrive to path. Dont know why, but else diffuser folder is not seen.
# os.environ['PYTHONPATH'] = str(os.environ['PYTHONPATH']) + '~/DiffuseDrive'

import diffuser.utils as utils
import torch
import wandb
# logging.basicConfig(filename='logs/training.log', encoding='utf-8', level=logging.INFO)
from logger_module import logger
from tqdm import tqdm

2023-07-27 20:16:13,291 - INFO - MUJOCO_GL is not set, so an OpenGL backend will be chosen automatically.
2023-07-27 20:16:13,301 - INFO - Successfully imported OpenGL backend: glfw
2023-07-27 20:16:13,357 - INFO - MuJoCo library version is: 2.3.5
pybullet build time: May 20 2022 19:45:31
/scratch_net/biwidl211/rl_course_10/.pyenv/versions/3.9.16/envs/playground_diffmodel/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_config = utils.Config(
    'datasets.CollectedSequenceDataset',
    savepath='dataset_config.pkl',
    env='carla-expert',
    horizon=12,
    normalizer='CDFNormalizer',
    preprocess_fns=[],
    use_padding=True,
    max_path_length=10,
    include_returns=True,
    returns_scale=400,
    discount=0.99,
    termination_penalty=-100,
    past_image_cond = True
)


dataset = dataset_config()

[ utils/config ] Imported diffuser.datasets:CollectedSequenceDataset

[utils/config ] Config: <class 'diffuser.datasets.sequence.CollectedSequenceDataset'>
    discount: 0.99
    env: carla-expert
    horizon: 12
    include_returns: True
    max_path_length: 10
    normalizer: CDFNormalizer
    past_image_cond: True
    preprocess_fns: []
    returns_scale: 400
    termination_penalty: -100
    use_padding: True

[ utils/config ] Saved config to: dataset_config.pkl



In [3]:
def cycle(dl):
    while True:
        for data in dl:
            yield data

In [4]:
dataloader = cycle(torch.utils.data.DataLoader(
    dataset, batch_size=1, num_workers=0, pin_memory=True, shuffle = False
))

In [5]:
tmp = next(dataloader)


/scratch_net/biwidl211/rl_course_10/.pyenv/versions/3.9.16/envs/playground_diffmodel/lib/python3.9/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
print(tmp)
print(tmp[0][0,:,:-1]) # x and y coordinates
print(tmp[0].shape)

Batch(trajectories=tensor([[[ 0.1866, -0.7555,  0.0481],
         [ 0.2406, -0.7969,  0.1485],
         [ 0.3397, -0.8768,  0.2052],
         [-1.0000, -1.0000, -1.0000],
         [ 0.2497, -0.7209, -0.0838],
         [ 0.2553, -0.4589, -0.1082],
         [ 0.0505, -0.2062, -0.0302],
         [-0.0202,  0.0422, -0.0122],
         [-0.0239,  0.2893, -0.0307],
         [ 0.0020,  0.5272, -0.0257],
         [ 0.0180,  0.7599, -0.0423],
         [ 0.0156,  0.9546, -0.0412]]]), conditions=tensor([[[ 0.1866, -0.7555,  0.0481],
         [ 0.2406, -0.7969,  0.1485],
         [ 0.3397, -0.8768,  0.2052],
         [-1.0000, -1.0000, -1.0000]]]), images=tensor([[[[[ 70.,  70.,  69.,  ..., 117., 117., 117.],
           [ 72.,  72.,  71.,  ..., 117., 117., 117.],
           [ 74.,  73.,  72.,  ..., 117., 117., 117.],
           ...,
           [  1.,   2.,   2.,  ...,  87.,  49.,  47.],
           [  2.,   2.,   3.,  ..., 136.,  84.,  67.],
           [  2.,   2.,   3.,  ..., 192., 131.,  69.]],

 

In [7]:
all_zeros_count = 0
waypoint_min = [[9999, 9999, 999] for i in range(tmp[0].shape[1])]
waypoint_max = [[-9999, -9999, -999] for i in range(tmp[0].shape[1])]
count = 0
for i, data in enumerate(tqdm(dataloader)):
    # print(i)
    # print(data[0].max())
    # print(data[0])
    if data[0][0,:,:-1].max() <= 1e-6:
        all_zeros_count += 1

    for _, waypoints in enumerate(data[0]):
        for j, waypoint in enumerate(waypoints):
            for k,coordinate in enumerate(waypoint):
                if coordinate < waypoint_min[j][k]:
                    waypoint_min[j][k] = coordinate.item()
                elif coordinate > waypoint_max[j][k]:
                    waypoint_max[j][k] = coordinate.item()
    
    if i > 5000:
        break

5001it [00:23, 213.32it/s]


In [14]:
print(all_zeros_count)
print(i)

26
101


In [ ]:
print(all_zeros_count/(i+1))

In [8]:
for waypoint in waypoint_min:
    print(waypoint)

[-0.0332845039665699, -1.0, -0.8062645792961121]
[0.1286248117685318, -1.0, -0.7206536531448364]
[0.30616500973701477, -1.0, -0.2907021939754486]
[-1.0, -1.0, -1.0]
[0.10142072290182114, -1.0, -0.6379013657569885]
[-0.2268529087305069, -0.9521424770355225, -1.0]
[-0.8609055280685425, -0.9287698864936829, -1.0]
[-1.0, -0.907497227191925, -1.0]
[-1.0, -0.8861192464828491, -1.0]
[-1.0, -0.8668367266654968, -1.0]
[-1.0, -0.8802236318588257, -1.0]
[-1.0, -0.9226405620574951, -1.0]


In [9]:
for waypoint in waypoint_max:
    print(waypoint)

[0.444726824760437, -0.23557864129543304, 0.9999999403953552]
[0.3731505274772644, -0.4062672555446625, 0.9999998807907104]
[0.3693992495536804, -0.6474551558494568, 0.6995053887367249]
[-1.0, -1.0, -1.0]
[0.39944735169410706, -0.5756507515907288, 0.47211360931396484]
[0.7157794833183289, -0.33090436458587646, 0.8020148873329163]
[0.9387130737304688, -0.15308807790279388, 0.96841961145401]
[0.927065908908844, 0.0478949211537838, 0.979634702205658]
[0.8704726099967957, 0.2893338203430176, 0.9493060111999512]
[0.8948606252670288, 0.5272664427757263, 0.9626799821853638]
[0.9187260270118713, 0.7599566578865051, 0.9482361674308777]
[0.9177979230880737, 0.9545958638191223, 0.9478774666786194]


# Load sampled data

In [1]:
import pandas as pd
import numpy as np
import os

csv_folder = "/scratch_net/biwidl211/rl_course_10/DiffuseDriveM/visualize_bev/deeplab_image+cmd_report/"

traj_std = np.array([   [0.3209, 4.0127, 0.1432],
                        [0.1167, 2.7227, 0.0992],
                        [0.0225, 1.3844, 0.0514],
                        [0.0000, 0.0000, 0.0000],
                        [0.1156, 1.3830, 0.0515],
                        [0.3243, 2.7180, 0.0999],
                        [0.6107, 4.0036, 0.1449],
                        [0.9655, 5.2458, 0.1867],
                        [1.3822, 6.4494, 0.2254],
                        [1.8558, 7.6204, 0.2616],
                        [2.3817, 8.7654, 0.2957],
                        [2.9568, 9.8893, 0.3280]])

In [2]:
l2_full_list = []
for file_path in os.listdir(csv_folder):
    if file_path.endswith('.csv'):
        print(file_path)
        df = pd.read_csv(csv_folder+file_path)
        l2_list = []
        for i in range(int((len(df.columns)-1)/3)):
            x = df['x'+str(i)]
            y = df['y'+str(i)]
            l2_time = np.array([])
            for j in range(len(df.index) - 1):
                l2 = np.sqrt(((x[0]-x[j+1])*traj_std[i][0])**2 + ((y[0] - y[j+1])*traj_std[i][1]) **2)
                l2_time = np.append(l2_time, l2)
            l2_list.append(l2_time)
        l2_full_list.append(l2_list)
#         break

result150000b46.csv
result150000b57.csv
result150000b84.csv
result150000b52.csv
result150000b85.csv
result150000b8.csv
result150000b47.csv
result150000b67.csv
result150000b65.csv
result150000b80.csv
result150000b89.csv
result150000b7.csv
result150000b83.csv
result150000b33.csv
result150000b35.csv
result150000b95.csv
result150000b55.csv
result150000b98.csv
result150000b96.csv
result150000b26.csv
result150000b24.csv
result150000b5.csv
result150000b13.csv
result150000b63.csv
result150000b4.csv
result150000b40.csv
result150000b14.csv
result150000b69.csv
result150000b20.csv
result150000b87.csv
result150000b38.csv
result150000b54.csv
result150000b97.csv
result150000b77.csv
result150000b64.csv
result150000b34.csv
result150000b37.csv
result150000b92.csv
result150000b91.csv
result150000b48.csv
result150000b10.csv
result150000b59.csv
result150000b66.csv
result150000b79.csv
result150000b99.csv
result150000b18.csv
result150000b28.csv
result150000b31.csv
result150000b70.csv
result150000b12.csv
resu

In [3]:
int((len(df.columns)-1)/3)
print(len(df.index))

11


In [4]:
print(len(l2_full_list))
overall_sum = np.array([0] * 12)
for data in l2_full_list:
    print(np.mean(data[4]))
    for i in range(12):
        overall_sum[i] += (data[i].mean())
    

100
0.17192087645139115
0.12240340435208283
0.04737263594646298
0.07312910737777464
0.018940012437513736
0.09112715474092822
0.07952984208627187
0.1431509516225085
0.9138948236368745
0.0015509379512073885
0.0314198474191451
0.03844799625722162
0.04777471186771341
1.4884037629814137
0.09647080403169693
0.11169716852784735
0.001485813220115869
0.08093714340594385
0.019114052892463946
0.0007879562217628682
0.006274650622347737
0.04150976564369717
0.002886895911029243
0.012103886182444469
0.055106640871516496
0.48721754646054577
0.1292768384963506
0.8798906070795554
0.15605485324686763
0.029664356000590704
0.3275148265048128
0.001457853018304702
0.02310660675896189
0.07073884314497582
0.0016788152252246863
0.5201834486380167
0.016366259315813765
0.07592011727980481
0.0344078908661347
0.12038904162928828
0.08330629028566193
0.05219640180950602
0.03596608884192625
1.7270354795216438
0.3322439749773458
0.09409824786652796
0.0011613773097234475
0.045748395816424495
0.004540029235902704
1.59625

In [5]:
print(overall_sum/len(l2_full_list))

[0.   0.   0.   0.   0.06 0.39 0.94 1.65 2.52 3.5  4.52 5.51]
